### Settings ISI project path as the absolute path, so that import functions will work for the files.

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

### Generating a list of class_names so that will be used to train and test model

In [ ]:
from utils.preprocess import get_dataloaders

train_loader, test_loader, class_names = get_dataloaders()
print(class_names)  # ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']


### Training model by settings the epoch value to 10, it can be increased to make it more efficient

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from models.cnn_model import EmotionCNN
from utils.preprocess import get_dataloaders

# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader, test_loader, class_names = get_dataloaders()

# Initialize model
model = EmotionCNN(num_classes=7).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train
EPOCHS = 10
for epoch in range(EPOCHS):
    model.train()
    running_loss, correct, total = 0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {running_loss:.4f} | Accuracy: {acc:.2f}%")

torch.save(model.state_dict(), "../models/emotion_cnn.pth")

Epoch 1/50 | Loss: 1482.0131 | Accuracy: 34.41%
Epoch 2/50 | Loss: 1300.6688 | Accuracy: 44.06%
Epoch 3/50 | Loss: 1203.7961 | Accuracy: 48.15%
Epoch 4/50 | Loss: 1124.8808 | Accuracy: 51.96%
Epoch 5/50 | Loss: 1054.7423 | Accuracy: 55.71%
Epoch 6/50 | Loss: 990.4159 | Accuracy: 58.67%
Epoch 7/50 | Loss: 926.5221 | Accuracy: 61.51%
Epoch 8/50 | Loss: 867.1499 | Accuracy: 64.15%
Epoch 9/50 | Loss: 807.8240 | Accuracy: 67.03%
Epoch 10/50 | Loss: 750.0900 | Accuracy: 69.33%
Epoch 11/50 | Loss: 698.3941 | Accuracy: 71.36%
Epoch 12/50 | Loss: 643.6269 | Accuracy: 73.71%
Epoch 13/50 | Loss: 592.2962 | Accuracy: 76.00%
Epoch 14/50 | Loss: 542.3959 | Accuracy: 78.07%
Epoch 15/50 | Loss: 496.6200 | Accuracy: 79.83%
Epoch 16/50 | Loss: 447.7706 | Accuracy: 81.97%
Epoch 17/50 | Loss: 408.7005 | Accuracy: 83.76%
Epoch 18/50 | Loss: 370.8716 | Accuracy: 85.23%
Epoch 19/50 | Loss: 333.0096 | Accuracy: 86.94%
Epoch 20/50 | Loss: 298.3669 | Accuracy: 88.12%
Epoch 21/50 | Loss: 264.8516 | Accuracy: 89.